<a href="https://colab.research.google.com/github/dataforgoodfr/batch8_mednum/blob/master/notebooks/acces_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Liste des médiathèques


In [19]:
# get mediatheques map
import pandas as pd
import numpy as np

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
ls

drive/  sample_data/


In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/carte-des-bibliotheques.csv', header=0, sep=';')
print(df.shape)
df.head()

(844, 21)


,Nom structure,Adresse,Code postal,Département,Ville,Année(s) subvention Région,Montant subvention 2000,Montant subvention 2001,Montant subvention 2002,Montant subvention 2003,Montant subvention 2004,Montant subvention 2005,Montant subvention 2006,Montant subvention 2007,Montant subvention 2008,Montant subvention 2009,Montant subvention 2010,Montant subvention 2011,Montant subvention 2012,Montant subvention 2013,wgs84
0,MÉDIATHÈQUE SAINT-JOHN PERSE,2 rue Édouard Poisson,93300,93,AUBERVILLIERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.9104747,2.385008"
1,MÉDIATHÈQUE GEORGES PÉREC,8-10 rue du Marqis de Raies,91080,91,COURCOURONNES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6293899,2.4135832"
2,MÉDIATHÈQUE LES MUSES,101 rue de la Division Leclerc,91310,91,LINAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6305113,2.2650636"
3,MÉDIATHÈQUE MUNICIPALE,2 route de la Creusée,91530,91,LE-VAL-SAINT-GERMAIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.5657217,2.0601725"
4,BIBLIOTHÈQUE LE GRENIER,5 place des Droits de l'Homme,91800,91,BOUSSY-SAINT-ANTOINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"48.6891076,2.5297452"


In [5]:
df = df[['Code postal', 'Ville', 'Département']]
df['Code postal'] = df['Code postal'].astype(str)
df.head()

,Code postal,Ville,Département
0,93300,AUBERVILLIERS,93
1,91080,COURCOURONNES,91
2,91310,LINAS,91
3,91530,LE-VAL-SAINT-GERMAIN,91
4,91800,BOUSSY-SAINT-ANTOINE,91


In [6]:
# mapping code postal - code INSEE
mapping = pd.read_csv('/content/drive/My Drive/Colab Notebooks/correspondances-code-insee-code-postal.csv', header=0, sep=';')
print(mapping.shape)
mapping.rename(columns={'Code Postal': 'Code postal'}, inplace=True)
mapping.head()

(1300, 17)


,Code INSEE,Code postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,ID Geofla,Code Commune,Code Canton,Code Arrondissement,Code Département,Code Région
0,91645,91370,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,Commune simple,121.0,999.0,15.5,"48.75044312,2.25171297214","{""type"": ""Polygon"", ""coordinates"": [[[2.238024...",16275,645,3,3,91,11
1,77133,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,Commune simple,88.0,1082.0,0.2,"48.4125606521,3.05294050556","{""type"": ""Polygon"", ""coordinates"": [[[3.076046...",31428,133,20,3,77,11
2,91378,91730,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,Commune simple,150.0,313.0,0.3,"48.5272680908,2.19718165044","{""type"": ""Polygon"", ""coordinates"": [[[2.203466...",30975,378,9,1,91,11
3,77243,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,Chef-lieu canton,71.0,579.0,20.2,"48.8730701858,2.70978081313","{""type"": ""Polygon"", ""coordinates"": [[[2.727542...",17000,243,14,5,77,11
4,94003,94110,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,Chef-lieu canton,70.0,232.0,19.5,"48.8058803597,2.33351024984","{""type"": ""Polygon"", ""coordinates"": [[[2.343851...",32123,3,34,3,94,11


In [ ]:
mapping.drop_duplicates(inplace=True)

In [ ]:
df.merge(mapping[['Code INSEE', 'Code postal']], how='inner', on='Code postal').drop_duplicates()  

,Code postal,Ville,Département,Code INSEE
0,93300,AUBERVILLIERS,93,93001
4,91080,COURCOURONNES,91,91182
5,91310,LINAS,91,91333
6,91310,LINAS,91,91425
7,91310,LINAS,91,91339
...,...,...,...,...
2722,77440,ARMENTIÈRES-EN-BRIE,77,77490
2723,77440,ARMENTIÈRES-EN-BRIE,77,77231
2724,77440,ARMENTIÈRES-EN-BRIE,77,77235
2725,77200,TORCY,77,77468


## 2. Liste des centres sociaux

In [ ]:
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib import request

import time


def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()        
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result    
    return timed

In [ ]:
@timeit
def scrap_table_page(url='https://www.senacs.fr/structure/csx?page=', page_number=1):
    """
    Generate a request on senacs page for a given 
    page number to get the name and location of 
    each social center.
    """

    url_page = url + str(page_number)
    request_text = request.urlopen(url_page).read()
    page = BeautifulSoup(request_text, 'html.parser')

    table = page.find('table', {'class': 'table table-striped table-hover'}).findAll('td')

    # fill the values of id, name and city
    ids = [id.getText() for id in table[::4]]
    names = [name.getText() for name in table[1::4]]
    cities = [city.getText() for city in table[2::4]]

    df = pd.DataFrame({'id': ids, 
                      'nom': names,
                      'ville': cities}
                      )
    df['page'] = page_number

    return df


def stack_pages(pages=range(91)):
    """
    Concatenate all empruntis scrapped data per region.
    """

    data = pd.DataFrame()    # empty Dataframe to fill
    for page_number in pages:
        temp = scrap_table_page(page_number=page_number)
        data = pd.concat([data, temp], axis=0)

    return data

In [ ]:
data = stack_pages()

In [ ]:
data['code_postal'] = data['ville'].apply(lambda x: x.split('-')[0])
data['ville'] = data['ville'].apply(lambda x: x.split('-')[1])

In [ ]:
data

,id,nom,ville,page,code_postal
0,91,CENTRE SOCIAL DES GRANDES BORNES,Goussainville,0,95190
1,92,CENTRE SOCIAL ARCHIPELIA,PARIS 20EME ARRONDISSEMENT,0,75020
2,93,Espace Socioculturel Val de Charente,Ruffec,0,16700
3,94,CENTRE SOCIAL Maison Des Habitants Champaret,Bourgoin,0,38300
4,95,CENTRE SOCIAL MAISON DE LA CHALLE,Éragny,0,95610
...,...,...,...,...,...
6,16264,CENTRE SOCIOCULTUREL MUNICIPAL DE SAINT-DIZIER,Saint,90,52100
7,16267,ESPACE DE VIE SOCIALE L.E.P.H.A.R.E,Aniche,90,59580
8,16351,ASSOCIATION FERME BEAUREPAIRE,Boulogne,90,62200
9,16355,OFFICE DE LA JEUNESSE,Bruay,90,62700


## 3. Distance des communes à l'accès le plus proche


In [2]:
# copy geoson file with every 'Franceservices' locations

import os

if 'COLAB_GPU' in os.environ:  # this is always set on Colab, the value is 0 or 1 depending on whether a GPU is attached
    from google.colab import auth
    #auth.authenticate_user()

    !rm -rf France-services/ 
    !git clone https://github.com/cget-carto/France-services.git
    !mv France-services/data/france_services.geojson .

Cloning into 'France-services'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 260 (delta 108), reused 221 (delta 78), pack-reused 0
Receiving objects: 100% (260/260), 2.12 MiB | 14.29 MiB/s, done.
Resolving deltas: 100% (108/108), done.


In [3]:
import pandas as pd
import json

with open("france_services.geojson", "r") as read_file: 
    fs = json.load(read_file) 

In [4]:
data_fs = pd.DataFrame()
for n, temp in enumerate(fs['features']): 
 
    df_temp = pd.DataFrame.from_records([{'latitude': temp['properties']['LATITUDE'], 
                            'longitude': temp['properties']['LATITUDE'], 
                            'departement': temp['properties']['DEPARTEMENT'],
                            'insee_com': temp['properties']['insee_com'],
                            'code_postal': temp['properties']['code_postal'], 
                            'lib_france_services': temp['properties']['lib_france_services']
                            }])
    
    data_fs = pd.concat([data_fs, df_temp], axis=0)

In [5]:
data_fs

,latitude,longitude,departement,insee_com,code_postal,lib_france_services
0,46.105747,46.105747,1,01033,01200,Valserhône
0,44.031548,44.031548,12,12197,12170,Réquista « Le Bercail »
0,44.475185,44.475185,12,12138,12330,Conques-Marcillac
0,49.291400,49.291400,14,14514,14130,Terre d’Auge
0,49.126169,49.126169,14,14057,14370,Val ès Dunes
...,...,...,...,...,...,...
0,47.617932,47.617932,89,89368,89520,Saint-Sauveur-en-Puisaye
0,48.980007,48.980007,95,95219,95120,Ermont
0,47.181000,47.181000,25,25527,25410,Saint Vit
0,43.208769,43.208769,31,31375,31310,Volvestre – Antenne Montesquieu


In [6]:
!pip install pgeocode

### Calculer la distance entre le centre des communes et la prefecture, sous-prefecture ou france-services la plus proche en km. 

- scrapper prefecture / sous-prefecture https://fr.wikipedia.org/wiki/Liste_des_pr%C3%A9fectures_de_France
- obtenir coordonnées des communes: OK via codes postaux + librairie `pgecode`

In [7]:
import pgeocode
dist = pgeocode.GeoDistance('fr')

!rm -rf batch8_mednum/
!git clone https://github.com/dataforgoodfr/batch8_mednum.git
!mv batch8_mednum/data/table_iris_insee_epci.csv .
!ls

Cloning into 'batch8_mednum'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 99 (delta 48), reused 22 (delta 5), pack-reused 0
Unpacking objects: 100% (99/99), done.
batch8_mednum	 france_services.geojson  table_iris_insee_epci.csv
France-services  sample_data


In [8]:
#data['distance'] = dist.query_postal_code(data[''].values, data[''].values)

In [12]:
#commune = pd.read_csv('table_iris_insee_epci.csv')
#del commune['Unnamed: 0']
commune = pd.read_csv('/content/drive/My Drive/Colab Notebooks/laposte_hexasmal.csv', delimiter=';')
commune.rename(columns={'Code_postal': 'code_postal'}, inplace=True)
commune

,Code_commune_INSEE,Nom_commune,code_postal,Ligne_5,Libellé_d_acheminement,coordonnees_gps
0,01004,AMBERIEU EN BUGEY,1500,NaN,AMBERIEU EN BUGEY,"45.9608475114,5.3729257777"
1,01008,AMBUTRIX,1500,NaN,AMBUTRIX,"45.9367134524,5.3328092349"
2,01015,ARBOYS EN BUGEY,1300,ST BOIS,ARBOYS EN BUGEY,"45.7237621545,5.65263086429"
3,01017,ARGIS,1230,NaN,ARGIS,"45.9337182132,5.48251100314"
4,01019,ARMIX,1510,NaN,ARMIX,"45.8541838459,5.58357838695"
...,...,...,...,...,...,...
39187,98824,POUEBO,98824,NaN,POUEBO,NaN
39188,98825,POUEMBOUT,98825,NaN,POUEMBOUT,NaN
39189,98828,SARRAMEA,98882,NaN,SARRAMEA,NaN
39190,98830,TOUHO,98831,NaN,TOUHO,NaN


In [ ]:
# calculer le france-service (ou prefecture, sous-prefecture etc.) le plus proche de chaque commune et bueno 

In [13]:
commune.dtypes
commune['code_postal'] = commune['code_postal'].astype(str)

In [14]:
def df_crossjoin(df1, df2, **kwargs):
    """
    Make a cross join (cartesian product) between two dataframes by using a constant temporary key.
    Also sets a MultiIndex which is the cartesian product of the indices of the input dataframes.
    See: https://github.com/pydata/pandas/issues/5401
    :param df1 dataframe 1
    :param df1 dataframe 2
    :param kwargs keyword arguments that will be passed to pd.merge()
    :return cross join of df1 and df2
    """
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1

    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey', axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))

    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)

    return res

df = df_crossjoin(commune[['code_postal']], data_fs[['code_postal']], suffixes=('_communes', '_fs'))
df = df.reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [38]:
df

,level_0,level_1,code_postal_communes,code_postal_fs
0,0,0,1500,01200
1,0,0,1500,12170
2,0,0,1500,12330
3,0,0,1500,14130
4,0,0,1500,14370
...,...,...,...,...
35586331,39191,0,98799,89520
35586332,39191,0,98799,95120
35586333,39191,0,98799,25410
35586334,39191,0,98799,31310


In [41]:
df.tail(50).apply(lambda row: dist.query_postal_code(row['code_postal_communes'], row['code_postal_fs']), axis=1)

# RAJOUTER 0 devant code postal à 4 chiffres

35586286    10979.659756
35586287    10979.659756
35586288    10979.659756
35586289    10979.782742
35586290    10929.637236
35586291    10929.334877
35586292    10871.425854
35586293    10961.396595
35586294    10879.484539
35586295    11075.435423
35586296    11066.282672
35586297    11101.031954
35586298    11041.421562
35586299    10705.990751
35586300    10721.849721
35586301    10695.177065
35586302    10740.956522
35586303    10729.962975
35586304    10728.142028
35586305    10713.622841
35586306    10716.912218
35586307    10706.375813
35586308    10618.072580
35586309    10563.378863
35586310    10658.372877
35586311    10635.267409
35586312    10850.982186
35586313    10873.315065
35586314    10884.024490
35586315    10836.028262
35586316    10871.801304
35586317    10764.382037
35586318    10827.776868
35586319    10827.776868
35586320    10809.971160
35586321    10800.703900
35586322    10679.434671
35586323    10686.112073
35586324    10678.352254
35586325    10682.352206


In [ ]:
df['distance'] = dist.query_postal_code(df['code_postal_communes'].values, data['code_postal_fs'].values)

## 4. Part des moins de 25 ans

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
demo1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_COM.csv', sep=';')
demo2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_ENSQP.csv', sep=';')
demo3 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_EPCI15.csv', sep=';')
demo4 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_EPCI19.csv', sep=';')
demo5 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data_DEMO_2020_V1_QP.csv', sep=';')

demo = pd.concat([demo1,demo2,demo3,demo4,demo5], axis=0).reset_index()
demo = demo[['CODGEO', 'POP_MUN', 'TX_TOT_0A24']]
demo

,CODGEO,POP_MUN,TX_TOT_0A24
0,01004,14359,"32,5"
1,01031,3646,"32,1"
2,01034,8909,"29,5"
3,01053,40490,"31,3"
4,01160,9236,"29,1"
...,...,...,...
2948,QP974045,1998,NaN
2949,QP974046,1411,NaN
2950,QP974047,2349,NaN
2951,QP974048,1798,NaN


In [20]:
#pd.merge(commune, demo, how='outer', on='')